In [1]:
# Have to patch sys.path until we have tighter taucmdr/anaconda integration.
# Need to move towards taucmdr as a conda package.
import os
import sys
try:
    import taucmdr
except ImportError:
    sys.path.insert(0, os.path.join(os.environ['__TAUCMDR_HOME__'], 'packages'))
finally:
    from taucmdr.model.project import Project

In [2]:
%pylab
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import pandas as pd
import math
import numpy as np
import operator

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [11]:
num_trials = Project.selected().experiment().num_trials
trials = Project.selected().experiment().trials(xrange(0, num_trials))
trial_data = {}
for i in xrange(0, num_trials):
    trial_data[i] = trials[i].get_data()

In [4]:
#trial0_intervals = trial0_data[0][0][0].interval_data()
#trial0_intervals.plot(kind='bar')
#plt.show()

In [169]:
trial_intervals = []
for trial in xrange(0, num_trials):
    for i in xrange(0, len(trial_data[trial])):
        for j in xrange(0, len(trial_data[trial][i])):
            for k in xrange(0, len(trial_data[trial][i][j])):
                trial_intervals.append(trial_data[trial][i][j][k].interval_data())
                
expr_intervals = pd.concat(trial_intervals)
#expr_intervals.xs('.TAU application', level=4)
expr_intervals.loc[(0,0,0,0)]


,Exclusive,Group,Inclusive,Call,ProfileCalls,Subcalls
.TAU application,8.0,TAU_USER,4453569.0,1,0,1
".TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}]",60.0,TAU_CALLPATH|TAU_DEFAULT,4453561.0,1,0,3
".TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}] => MPI_Finalize()",318.0,TAU_CALLPATH|MPI,318.0,1,0,0
".TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}] => MPI_Init()",2880.0,TAU_CALLPATH|MPI,2880.0,1,0,0
".TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}] => double do_work(void) C [{matmult.c} {132,1}-{168,1}]",65.0,TAU_CALLPATH|TAU_USER,4450303.0,1,0,11
".TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}] => double do_work(void) C [{matmult.c} {132,1}-{168,1}] => double **allocateMatrix(int, int) C [{matmult.c} {42,1}-{49,1}]",2163.0,TAU_CALLPATH|TAU_USER,2163.0,3,0,0
".TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}] => double do_work(void) C [{matmult.c} {132,1}-{168,1}] => void compute(double **, double **, double **, int, int, int) C [{matmult.c} {90,1}-{109,1}]",2293036.0,TAU_CALLPATH|TAU_USER,2336044.0,1,0,100001
".TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}] => double do_work(void) C [{matmult.c} {132,1}-{168,1}] => void compute(double **, double **, double **, int, int, int) C [{matmult.c} {90,1}-{109,1}] => double multiply(double, double) C [{matmult.c} {59,1}-{61,1}]",43008.0,TAU_CALLPATH|TAU_USER,43008.0,100001,0,0
".TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}] => double do_work(void) C [{matmult.c} {132,1}-{168,1}] => void compute_interchange(double **, double **, double **, int, int, int) C [{matmult.c} {111,1}-{130,1}]",2108681.0,TAU_CALLPATH|TAU_USER,2108681.0,1,0,0
".TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}] => double do_work(void) C [{matmult.c} {132,1}-{168,1}] => void freeMatrix(double **, int, int) C [{matmult.c} {51,1}-{57,1}]",114.0,TAU_CALLPATH|TAU_USER,114.0,3,0,0


In [198]:
# levels: 0=trial, 1=node, 2=context, 3=thread, 4=region name

from taucmdr.error import InternalError

def largest_stddev(dfs,n):
    return dfs['Exclusive'].groupby(level=3).std(ddof=0).dropna().sort_values(ascending=False, axis=0)[:n]

def compute_correlation(group1, group2):
    return np.corrcoef(group1, group2)[0,1]

def largest_correlation(dfs,n):
    #regions_dict = {}
    #for region in regions:
    #    regions_dict[region] = []
    #    for tr in dfs:
    #        for rank in dfs[tr]:
    #            for context in dfs[tr][rank]:
    #                for thread in dfs[tr][rank][context]:
    #                    try:
    #                        regions_dict[region].append(dfs[tr][rank][context][thread].at[region, 'Exclusive'])
    #                    except:
    #                        regions_dict[region].append(0)
    #regions_dict['.TAU application'] = []
    #for tr in dfs:
    #    for rank in dfs[tr]:
    #        for context in dfs[tr][rank]:
    #            for thread in dfs[tr][rank][context]:
    #                regions_dict['.TAU application'].append(dfs[tr][rank][context][thread].at['.TAU application', 'Inclusive'])
    #corr_coef = {}
    #for region in regions:
    #    corr_coef[region] = np.corrcoef(regions_dict[region], regions_dict['.TAU application'])[0,1]
    #return [x for x in sorted(corr_coef.iteritems(), key=operator.itemgetter(1), reverse=True)[:n]]
    gr = dfs['Exclusive'].groupby(level=4)
    regions = dfs.index.get_level_values(4)
    regions_dict = {}
    for region,value in gr:
        #print 'region %s: %s' %(region, value)
        regions_dict[region] = value.tolist()
    corr_coef = {}
    for region in regions:
        corr_coef[region] = np.corrcoef(regions_dict[region], regions_dict['.TAU application'])[0,1]
    return [x for x in sorted(corr_coef.iteritems(), key=operator.itemgetter(1), reverse=True)[:n]]
    
#    gr = dfs['Exclusive'].groupby(level=4)
#    #gr.get_group('.TAU application')
#    #print gr.agg(lambda x: compute_correlation(x, gr.get_group('.TAU application')))
#    for i,group in gr:
#        #print '%s: %s' %(i, np.corrcoef(group, gr.get_group('.TAU application'))[0,1])
#        print i
#        #print group[1]
#        print group
#        #print group

def largest_exclusive(df,n):
    rows = df.nlargest(n, 'Exclusive')
    return rows['Exclusive']

def hotspots(dfs, n, flag):
    if flag == 0:
        largest = largest_exclusive(dfs,n)
    elif flag == 1:
        largest = largest_stddev(dfs,n)
    elif flag == 2:
        largest = largest_correlation(dfs,n)
    else:
        print 'Invalid flag'
    y = ['exclusive time', 'standard deviation', 'correlation to total runtime']
    print 'Hotspot Analysis Summary'
    print '='*80
    print 'The code regions with largest %s are: ' %y[flag]
    for i in xrange(0,n):
        print '%s: %s (%s)' %(i+1, largest.index[i], largest[i])
    

hotspots(expr_intervals.loc[(0,0,0,0)], 5, 0)
hotspots(expr_intervals.loc[7], 5, 1)
#hotspots(expr_intervals, 5, 2)

Hotspot Analysis Summary
The code regions with largest exclusive time are: 
1: .TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}]   => double do_work(void) C [{matmult.c} {132,1}-{168,1}]   => void compute(double **, double **, double **, int, int, int) C [{matmult.c} {90,1}-{109,1}]   (2293036.0)
2: void compute(double **, double **, double **, int, int, int) C [{matmult.c} {90,1}-{109,1}]   (2293036.0)
3: .TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}]   => double do_work(void) C [{matmult.c} {132,1}-{168,1}]   => void compute_interchange(double **, double **, double **, int, int, int) C [{matmult.c} {111,1}-{130,1}]   (2108681.0)
4: void compute_interchange(double **, double **, double **, int, int, int) C [{matmult.c} {111,1}-{130,1}]   (2108681.0)
5: .TAU application => int main(int, char **) C [{matmult.c} {209,1}-{299,1}]   => double do_work(void) C [{matmult.c} {132,1}-{168,1}]   => void compute(double **, double **, double **,